<a href="https://colab.research.google.com/github/anicelysantos/book-python-para-analise-de-dados/blob/main/agregacao_operacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [30]:
import pandas as pd
import numpy as np

# Funcionamento do GroupBy

In [31]:
df = pd.DataFrame({'key1':['a','a','b','b','a'], 'key':['one','two','one','two','one'], 'data1': np.random.randn(5), 'data2': np.random.randn(5)})
df

,key1,key,data1,data2
0,a,one,1.429162,-0.165951
1,a,two,0.165850,0.338924
2,b,one,0.561908,-1.167690
3,b,two,-0.735036,0.475993
4,a,one,-1.172019,0.844997


In [32]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [33]:
grouped.mean()

key1
a    0.140998
b   -0.086564
Name: data1, dtype: float64

In [34]:
means = df['data1'].groupby([df['key1'], df['key']]).mean()
means

key1  key
a     one    0.128572
      two    0.165850
b     one    0.561908
      two   -0.735036
Name: data1, dtype: float64

In [35]:
means.unstack()

key,one,two
key1,,
a,0.128572,0.165850
b,0.561908,-0.735036


In [36]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])
df['data1'].groupby([states, years]).mean()

California  2005    0.165850
            2006    0.561908
Ohio        2005    0.347063
            2006   -1.172019
Name: data1, dtype: float64

In [37]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.140998,0.339323
b,-0.086564,-0.345849


In [38]:
df.groupby(['key1', 'key']).mean()

data1     data2
key1 key                    
a    one  0.128572  0.339523
     two  0.165850  0.338924
b    one  0.561908 -1.167690
     two -0.735036  0.475993

In [39]:
df.groupby(['key1', 'key']).size()

key1  key
a     one    2
      two    1
b     one    1
      two    1
dtype: int64

**Iterando por grupos**

In [40]:
for nome, grupo in df.groupby('key1'):
  print(nome)
  print(grupo)

a
  key1  key     data1     data2
0    a  one  1.429162 -0.165951
1    a  two  0.165850  0.338924
4    a  one -1.172019  0.844997
b
  key1  key     data1     data2
2    b  one  0.561908 -1.167690
3    b  two -0.735036  0.475993


In [41]:
for (k1,k2), grupo in df.groupby(['key1','key']):
  print((k1,k2))
  print(grupo)

('a', 'one')
  key1  key     data1     data2
0    a  one  1.429162 -0.165951
4    a  one -1.172019  0.844997
('a', 'two')
  key1  key    data1     data2
1    a  two  0.16585  0.338924
('b', 'one')
  key1  key     data1    data2
2    b  one  0.561908 -1.16769
('b', 'two')
  key1  key     data1     data2
3    b  two -0.735036  0.475993


In [42]:
#gerando dicionario com uma parte dos dados

parte = dict(list(df.groupby('key1')))
parte['b']

,key1,key,data1,data2
2,b,one,0.561908,-1.167690
3,b,two,-0.735036,0.475993


In [43]:
df.dtypes

key1      object
key       object
data1    float64
data2    float64
dtype: object

In [44]:
grouped = df.groupby(df.dtypes, axis=1)
for dtype, grupo in grouped:
  print(dtype)
  print(grupo)

float64
      data1     data2
0  1.429162 -0.165951
1  0.165850  0.338924
2  0.561908 -1.167690
3 -0.735036  0.475993
4 -1.172019  0.844997
object
  key1  key
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


**Selecionando uma coluna ou um subconjunto de colunas**

In [45]:
df.groupby(['key1', 'key'])[['data2']].mean()

data2
key1 key          
a    one  0.339523
     two  0.338924
b    one -1.167690
     two  0.475993

In [46]:
s_grouped = df.groupby(['key1', 'key'])['data2']
s_grouped

In [47]:
s_grouped.mean()

key1  key
a     one    0.339523
      two    0.338924
b     one   -1.167690
      two    0.475993
Name: data2, dtype: float64

**Agrupando com dicionário e séries**

In [48]:
people = pd.DataFrame(np.random.randn(5,5), columns=['a','b','c','d','e'], index=['Joe', 'Steve','Wes','Jim','Travis'])

In [49]:
people.iloc[2:3, [1,2]] = np.nan
people

,a,b,c,d,e
Joe,-1.614674,1.327892,0.346682,0.036614,-0.159925
Steve,-0.510372,-0.244310,0.072814,0.313484,-0.360383
Wes,0.303616,NaN,NaN,-0.553217,2.547411
Jim,-0.717671,-1.604341,2.123884,-2.822483,-0.858518
Travis,0.029933,0.604866,0.580408,-1.325552,0.093526


In [50]:
#somar as colunas por grupo
mapping = {'a': 'red', 'b':'red', 'c':'blue', 'd':'blue','e':'red','f':'orange'}

In [51]:
by_columns = people.groupby(mapping, axis=1)
by_columns.sum()

,blue,red
Joe,0.383296,-0.446707
Steve,0.386298,-1.115065
Wes,-0.553217,2.851026
Jim,-0.698599,-3.180530
Travis,-0.745144,0.728326


In [52]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [53]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


**Agrupando com funções**

In [54]:
#agrupando por tamanho de string (contando string do índice)
people.groupby(len).sum()

,a,b,c,d,e
3,-2.028730,-0.276449,2.470565,-3.339086,1.528968
5,-0.510372,-0.244310,0.072814,0.313484,-0.360383
6,0.029933,0.604866,0.580408,-1.325552,0.093526


In [55]:
key_list = ['one', 'one', 'one','two','two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.614674  1.327892  0.346682 -0.553217 -0.159925
  two -0.717671 -1.604341  2.123884 -2.822483 -0.858518
5 one -0.510372 -0.244310  0.072814  0.313484 -0.360383
6 two  0.029933  0.604866  0.580408 -1.325552  0.093526

**Agrupando por níveis de índice**

In [56]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'], [1,3,5,1,3]], names=['cty','tenor'])
hier_df = pd.DataFrame(np.random.randn(4,5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.093889  1.158818  0.178637 -0.893378  1.080049
1      0.158462 -0.838732 -0.311818 -0.164001  0.095608
2     -0.217977  1.324871  0.772960  0.184043 -0.168471
3      0.588206  1.080786  1.109897  0.779604 -0.470754

In [57]:
hier_df.groupby(level = 'cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


# Agregação de dados

In [58]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.176499
b    0.432214
Name: data1, dtype: float64

In [59]:
#Para usar uma função própria, utilize agg e passe a função entre parenteses
def peak_to_peak(arr):
  return arr.max() - arr.min()
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,2.601180,1.010949
b,1.296945,1.643683


In [60]:
grouped.describe()

data1                      ...     data2                    
     count      mean       std  ...       50%       75%       max
key1                            ...                              
a      3.0  0.140998  1.300768  ...  0.338924  0.591961  0.844997
b      2.0 -0.086564  0.917078  ... -0.345849  0.065072  0.475993

[2 rows x 16 columns]

**Aplicação de função nas colunas e aplicação de várias funções**

In [61]:
tips = pd.read_csv('/content/drive/MyDrive/dados_pandas/pydata-book-2nd-edition/examples/tips.csv')

In [62]:
#Acrescenta a porcentagem de gorjeta sobre o total da conta
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [63]:
grouped = tips.groupby(['day','smoker'])
grouped_pct = grouped['tip_pct']
#É possivel passar uma função descritiva como string
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [64]:
grouped_pct.agg(['mean','std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [65]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [66]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [67]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [68]:
#personalizar o nome das colunas
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [69]:
grouped.agg({'tip': np.max, 'size':'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [70]:
grouped.agg({'tip_pct':['min','max','mean','std'], 'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

**Devolvendo dados agregados sem índice de linha**

In [71]:
tips.groupby(['day','smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


# Método apply: separar-aplicar-combinar genérico

Usando a base de dados de gorjetas vamos selecionar os cinco primeiros valores de `tip_pct`.

In [72]:
#Essa função seleciona as linhas com os maiores valores em uma coluna
def top (df, n=5, column = 'tip_pct'):
  return df.sort_values(by=column)[-n:]

top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [73]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [74]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [75]:
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [76]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [77]:
#mesma coisa da célula anterior
f = lambda x: x.describe()
grouped.apply(f)

total_bill       tip  size   tip_pct
day  smoker                                            
Fri  No     count    4.000000  4.000000  4.00  4.000000
            mean    18.420000  2.812500  2.25  0.151650
            std      5.059282  0.898494  0.50  0.028123
            min     12.460000  1.500000  2.00  0.120385
            25%     15.100000  2.625000  2.00  0.137239
...                       ...       ...   ...       ...
Thur Yes    min     10.340000  2.000000  2.00  0.090014
            25%     13.510000  2.000000  2.00  0.148038
            50%     16.470000  2.560000  2.00  0.153846
            75%     19.810000  4.000000  2.00  0.194837
            max     43.110000  5.000000  4.00  0.241255

[64 rows x 4 columns]

**Suprimindo as chaves de grupo**

In [78]:
#ocultar os índices hierarquicos
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


**Análise de quantis e de buckets**

In [79]:
frame = pd.DataFrame({'data1': np.random.randn(1000), 'data2':np.random.randn(1000)})

In [80]:
quartiles = pd.cut(frame.data1, 4)

In [81]:
quartiles[:10]

0    (-2.069, -0.424]
1       (1.22, 2.865]
2    (-2.069, -0.424]
3      (-0.424, 1.22]
4      (-0.424, 1.22]
5      (-0.424, 1.22]
6       (1.22, 2.865]
7    (-2.069, -0.424]
8      (-0.424, 1.22]
9       (1.22, 2.865]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-3.72, -2.069] < (-2.069, -0.424] < (-0.424, 1.22] <
                                    (1.22, 2.865]]

In [82]:
def get_stats(group):
  return {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean':group.mean()}

In [83]:
grouped = frame.data2.groupby(quartiles)

In [85]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
"(-3.72, -2.069]",-1.257553,1.915877,25.0,0.068585
"(-2.069, -0.424]",-2.919177,2.297760,308.0,-0.018077
"(-0.424, 1.22]",-3.361145,3.794298,559.0,-0.018275
"(1.22, 2.865]",-2.198160,2.389064,108.0,0.121256


In [86]:
#Obter só o número dos quantis

grouping = pd.qcut(frame.data1, 10, labels = False)

In [87]:
grouped = frame.data2.groupby(grouping)

In [88]:
grouped.apply(get_stats).unstack()

,min,max,count,mean
data1,,,,
0,-2.635054,2.297760,100.0,-0.149312
1,-2.631397,2.248178,100.0,-0.013164
2,-1.957222,2.018304,100.0,0.139471
3,-3.151042,2.105593,100.0,-0.011486
4,-2.939606,2.427985,100.0,0.078950
5,-3.361145,1.885870,100.0,-0.041869
6,-2.124783,2.948816,100.0,-0.085251
7,-2.443917,3.794298,100.0,0.069613
8,-2.658777,2.432044,100.0,-0.106315


**Preenchendo valores ausentes com valores específicos de grupo**

In [89]:
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1    0.594843
2         NaN
3   -0.079868
4         NaN
5   -2.288693
dtype: float64

In [90]:
#preenchendo os valores ausentes com a media dos números
s.fillna(s.mean())

0   -0.591239
1    0.594843
2   -0.591239
3   -0.079868
4   -0.591239
5   -2.288693
dtype: float64

In [91]:
#preenchendo de acordo com o grupo
states = ['Ohio', 'New Your', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

In [92]:
group_key = ['East'] * 4 + ['West'] * 4

In [93]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -1.722827
New Your     -2.091112
Vermont      -0.072009
Florida       1.281357
Oregon       -2.906289
Nevada       -1.143890
California    0.019137
Idaho         1.826843
dtype: float64

In [94]:
#Definindo alguns valores como ausentes
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -1.722827
New Your     -2.091112
Vermont            NaN
Florida       1.281357
Oregon       -2.906289
Nevada             NaN
California    0.019137
Idaho              NaN
dtype: float64

In [95]:
#Preenchendo usando as medias dos grupos
fill_mean = lambda g: g.fillna(g.mean())

In [96]:
data.groupby(group_key).apply(fill_mean)

Ohio         -1.722827
New Your     -2.091112
Vermont      -0.844194
Florida       1.281357
Oregon       -2.906289
Nevada       -1.443576
California    0.019137
Idaho        -1.443576
dtype: float64

In [97]:
#Preencher com valores pré-definidos
fill_values = {'East': 0.5, 'West': -1}


In [98]:
fill_func = lambda g: g.fillna(fill_values[g.name])

In [99]:
data.groupby(group_key).apply(fill_func)

Ohio         -1.722827
New Your     -2.091112
Vermont       0.500000
Florida       1.281357
Oregon       -2.906289
Nevada       -1.000000
California    0.019137
Idaho        -1.000000
dtype: float64

**Amostragem aleatória e permutação**<br>

Uma maneira de construir um baralho em inglês

In [100]:
#Copas (Hearts), Espada (Spades), Paus(Clubs), Ouro (Diamonds)
suits = ['H', 'S', 'C', 'D']
card_val=(list(range(1,11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2,11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
  cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)

In [101]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [102]:
#sortear uma mão de cinco baralhos

def draw(deck, n=5):
  return deck.sample(n)

draw(deck)

8D      8
7H      7
8S      8
10D    10
QS     10
dtype: int64

In [104]:
#Duas cartas aleatórias de cada naipe
get_suit = lambda card: card[-1] #a ultima letra é o naipe
deck.groupby(get_suit).apply(draw, n=2)

C  7C     7
   2C     2
D  AD     1
   QD    10
H  9H     9
   4H     4
S  JS    10
   QS    10
dtype: int64

In [105]:
#Outra forma de fazer a mesma coisa da célula anterior
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

KC    10
2C     2
AD     1
QD    10
9H     9
3H     3
AS     1
7S     7
dtype: int64

**Média ponderada de grupos e correlação**